# Pivot Tables

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
titanic = sns.load_dataset('titanic')

In [3]:
# Helper Function to dusplay DataFrames in Horizontal 

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

* A *pivot table* is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data.
* The pivot table takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data.
* The difference between pivot tables and ``GroupBy`` can sometimes cause confusion; it helps me to think of pivot tables as essentially a *multidimensional* version of ``GroupBy`` aggregation.
* That is, you split-apply-combine, but both the split and the combine happen across not a one-dimensional index, but across a two-dimensional grid.

****

## Titanic Dataset Details

For the examples in this section, we'll use the database of passengers on the *Titanic*, available through the Seaborn library (see [Visualization With Seaborn](04.14-Visualization-With-Seaborn.ipynb)):

In [4]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


This contains a wealth of information on each passenger of that ill-fated voyage, including gender, age, class, fare paid, and much more.

****

## Using Groupby Function

* Like to get the suvival rate by gender. The same can be achived by using `groupby` function. 

In [7]:
titanic.groupby('sex').count()

,survived,pclass,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
sex,,,,,,,,,,,,,,
female,314,314,261,314,314,314,312,314,314,314,97,312,314,314
male,577,577,453,577,577,577,577,577,577,577,106,577,577,577


In [8]:
titanic.groupby('sex')['survived'].agg(['mean', 'sum', 'count'])

,mean,sum,count
sex,,,
female,0.742038,233,314
male,0.188908,109,577


* This immediately gives us some insight: overall, three of every four females on board survived, while only one in five males survived.

* We might like to go one step deeper and look at survival by both sex and, say, class. 
* Using the vocabulary of ``GroupBy``, we might proceed using something like this:
* we *group by* class and gender, *select* survival, *apply* a mean aggregate, *combine* the resulting groups, and then *unstack* the hierarchical index to reveal the hidden multidimensionality. In code:

In [9]:
titanic.groupby(['sex', 'class'])['survived'].agg(['mean', 'sum', 'count']).unstack(1)

mean                       sum              count             
class      First    Second     Third First Second Third First Second Third
sex                                                                       
female  0.968085  0.921053  0.500000    91     70    72    94     76   144
male    0.368852  0.157407  0.135447    45     17    47   122    108   347

* This gives us a better idea of how both gender and class affected survival, 
* This two-dimensional ``GroupBy`` is common enough that Pandas includes a convenience routine, ``pivot_table``, which succinctly handles this type of multi-dimensional aggregation.

****

## Pivot Table Syntax

Here is the equivalent to the preceding operation using the ``pivot_table`` method of ``DataFrames``:

```python
DataFrame.pivot_table(values=None, index=None, 
                      columns=None, aggfunc='mean', 
                      fill_value=None, margins=False, 
                      dropna=True, margins_name='All',
                      observed=False, sort=True)
```
Documentation Link for [DataFrame.pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html)

In [10]:
titanic.pivot_table(
    values=(['survived']), 
    index=['sex'], 
    columns=(['class']), 
    aggfunc=['mean','sum','count'],
).stack(1)

mean                       sum              count         \
class               First    Second     Third First Second Third First Second   
sex                                                                             
female survived  0.968085  0.921053  0.500000    91     70    72    94     76   
male   survived  0.368852  0.157407  0.135447    45     17    47   122    108   

                       
class           Third  
sex                    
female survived   144  
male   survived   347

This is eminently more readable than the ``groupby`` approach, and produces the same result.

****

### Multi-level pivot tables

Just as in the ``GroupBy``, the grouping in pivot tables can be specified with multiple levels, and via a number of options.
For example, we might be interested in looking at age as a third dimension.
We'll bin the age using the ``pd.cut`` function:

In [11]:
age = pd.cut(titanic['age'], [0, 20, 30, 40, 80])
titanic.pivot_table(
    values ='survived',
    index =['sex', age],
    columns='class')

class               First    Second     Third
sex    age                                   
female (0, 20]   0.928571  1.000000  0.510638
       (20, 30]  0.952381  0.892857  0.500000
       (30, 40]  1.000000  0.941176  0.428571
       (40, 80]  0.961538  0.846154  0.111111
male   (0, 20]   0.571429  0.526316  0.197368
       (20, 30]  0.473684  0.000000  0.144330
       (30, 40]  0.520000  0.115385  0.142857
       (40, 80]  0.280000  0.095238  0.064516

We can apply the same strategy when working with the columns as well; let's add info on the fare paid using ``pd.qcut`` to automatically compute quantiles:

In [12]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 20]               NaN  1.000000  0.680000          0.928571   
       (20, 30]              NaN  0.916667  0.500000          0.952381   
       (30, 40]              NaN  0.875000  0.200000          1.000000   
       (40, 80]              NaN  0.800000  0.333333          0.961538   
male   (0, 20]               NaN  0.142857  0.195652          0.571429   
       (20, 30]              NaN  0.000000  0.141176          0.473684   
       (30, 40]              0.0  0.153846  0.128205          0.619048   
       (40, 80]              NaN  0.153846  0.068966          0.280000   

fare                                 
class              Second     Third  
sex    age                           
female (0, 20]   1.000000  0.318182  
       (20, 30]  0.875000  0.500000  
       (30, 40]  1.000000  0.555556  
       (40, 80]  0.875000  0.000000  
male   (0, 20]   0.750000  0.200000  
       (20, 30]  0.000000  0.166667  
       (30, 40]  0.076923  0.200000  
       (40, 80]  0.000000  0.000000

***

### Additional pivot table options
* The ``aggfunc`` keyword controls what type of aggregation is applied, which is a mean by default.
* As in the GroupBy, the aggregation specification can be a string representing one of several common choices (e.g., ``'sum'``, ``'mean'``, ``'count'``, ``'min'``, ``'max'``, etc.) or a function that implements an aggregation (e.g., ``np.sum()``, ``min()``, ``sum()``, etc.).
* Additionally, it can be specified as a dictionary mapping a column to any of the above desired options

In [13]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Notice also here that we've omitted the ``values`` keyword; when specifying a mapping for ``aggfunc``, this is determined automatically.

At times it's useful to compute totals along each grouping.
This can be done via the ``margins`` keyword:

In [14]:
titanic.pivot_table(values='survived', index='sex', columns='class', aggfunc='count', margins=True, 
                    margins_name='Total')

class,First,Second,Third,Total
sex,,,,
female,94,76,144,314
male,122,108,347,577
Total,216,184,491,891


The margin label can be specified with the ``margins_name`` keyword, which defaults to ``"All"``.

***